In [1]:
#Building Face detection model

import os
import numpy as np
from PIL import Image
import cv2
import pickle

#baseDirectory  = os.path.dirname(os.path.abspath(__file__))

imageDirectory = os.path.dirname("DIRECTORY PATH FOR IMAGE DATASET")

face_cascade = cv2.CascadeClassifier('DIRECTORY PATH CONTAINING HAARCASCADE CLASSIFIERS/haarcascades/haarcascade_frontalface_alt2.xml')

recognizer =cv2.face.LBPHFaceRecognizer_create() #cv2.face.creaeLBPHFaceRecognizer()#

currentId = 0
label_ids = {}
yLabel = []
xTrain = []
        
for root , dirs , files in os.walk(imageDirectory):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root,file)
            label = os.path.basename(root).replace(" ","-").lower()
            #os.path.dirname(path)  can be written as root
            #print(label,path)

            if label in label_ids:
                pass
            else:
                label_ids[label] = currentId
                currentId+=1
            id_=label_ids[label]
            #print(label_ids)

           # yLabel.append(label)
            #xTrain.append(path)
            pilImage = Image.open(path).convert("L")
            size = (550, 550)
            final_image = pilImage.resize(size,Image.ANTIALIAS)
            imageArray = np.array(pilImage, "uint8")
            #print(imageArray)
            faces = face_cascade.detectMultiScale(imageArray, scaleFactor=1.5  ,minNeighbors = 5)

            #print(id_)
            for (x,y,w,h) in faces:
                roi = imageArray[y:y+h , x:x+h]
                xTrain.append(roi)
                yLabel.append(id_)

# print(yLabel)
# print(xTrain)

with open("labels.pickle", 'wb') as f:
    pickle.dump(label_ids , f)

recognizer.train(xTrain , np.array(yLabel))
recognizer.save("trainner.yml")

In [1]:
#Recognizing faces in a clip

import numpy as np 
import cv2
import pickle
import os
import time
import math

eye_cascade = cv2.CascadeClassifier('DIRECTORY PATH CONTAINING HAARCASCADE CLASSIFIERS/haarcascades/haarcascade_eye.xml')
face_cascade = cv2.CascadeClassifier('DIRECTORY PATH CONTAINING HAARCASCADE CLASSIFIERS/haarcascades/haarcascade_frontalface_alt2.xml')
recognizer =cv2.face.LBPHFaceRecognizer_create() #cv2.face.createLBPHFaceRecognizer()#
smile_cascade = cv2.CascadeClassifier('DIRECTORY PATH CONTAINING HAARCASCADE CLASSIFIERS/haarcascades/haarcascade_smile.xml')

actors=[]

s=0 #timer variable
#--------------Finding the length of the video in seconds--------------------------------
# cap.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
# length=cap.get(cv2.CAP_PROP_POS_MSEC)
# length=str(math.ceil(length))[0:3]
# length=int(length)
#print(length)
#----------------------------------------------------------------------------------------



recognizer.read("trainner.yml")
labels={"person_name": 1}

with open("labels.pickle", 'rb') as f:
    ogLabels = pickle.load(f)
    labels = {v:k for k,v in ogLabels.items()}

cap=cv2.VideoCapture("PATH OF THE TEST VIDEO FILE")
#cap =cv2.VideoCapture("C:/Users/heman/Downloads/Mission.Impossible.Fallout.2018.720p.HC.HDRip.HEVC.MkvCage.mkv")

total_frames=cap.get(cv2.CAP_PROP_FRAME_COUNT)
#print(total_frames)

while(True):
    
    current_frame=cap.get(cv2.CAP_PROP_POS_FRAMES)
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray , scaleFactor=1.5  ,minNeighbors = 5)
    for(x, y, w, h) in faces:
        #print(x,y,w,h)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        id_ , conf = recognizer.predict(roi_gray)
        #print(conf)
        #print(conf," ",labels[id_])
        if(labels[id_] not in actors ):
            actors.append(labels[id_])
        if (conf >= 4) and (conf <= 85):
            #print(id_)
            #print(labels[id_])
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            color=(255,255,255)
            stroke= 2
            cv2.putText(frame,name,(x,y), font ,1 ,color ,stroke ,cv2.LINE_AA)
        

        img_item = "7.png"
        cv2.imwrite(img_item,roi_gray)
        color = (72,120 ,0)
        stroke = 4
        cv2.rectangle(frame, (x,y) , (x+w, y+h),color , stroke) 
        subitems = smile_cascade.detectMultiScale(roi_gray)
        for(ex,ey,ew,eh) in subitems:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew , ey+eh) , (8,255,8) ,2)


    cv2.imshow('frame' ,frame)
    #cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES)
    if cv2.waitKey(20) & 0xFF ==ord('q'):
        break
    #print(current_frame)
    if (current_frame>=(total_frames-1)):
        break
        
cap.release()
cv2.destroyAllWindows()

#roi  region of interest

print(actors)

['tomcruise']


In [2]:
# AUDIO TO VIDEO CONVERSION

import moviepy.editor as mp
clip = mp.VideoFileClip("PATH OF THE TEST VIDEO FILE")
clip.audio.write_audiofile("NAME OF THE OUTPUT AUDIO FILE")

MoviePy - Writing audio in Intro.mp3


MoviePy - Done.


In [4]:
# USING IBM WATSON FOR AUDIO TO TEXT CONVERSION

import json
from os.path import join, dirname
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('API KEY')
speech_to_text = SpeechToTextV1(
    authenticator=authenticator
)

speech_to_text.set_service_url('https://gateway-lon.watsonplatform.net/speech-to-text/api')

class MyRecognizeCallback(RecognizeCallback):
    def __init__(self):
        RecognizeCallback.__init__(self)

    def on_data(self, data):
        print(json.dumps(data, indent=2))
        with open("FILE NAME IN WHICH OUTPUT WILL BE WRITTEN.json", "w+") as read_file:
            json.dump(data,read_file)
        
    def on_error(self, error):
        print('Error received: {}'.format(error))

    def on_inactivity_timeout(self, error):
        print('Inactivity timeout: {}'.format(error))

myRecognizeCallback = MyRecognizeCallback()

with open(join(dirname('conv'), './.', 'AUDIO FILE PATH'),
              'rb') as audio_file:
    audio_source = AudioSource(audio_file)
    speech_to_text.recognize_using_websocket(
        audio=audio_source,
        content_type='audio/mp3',
        recognize_callback=myRecognizeCallback,
        model='en-US_BroadbandModel',
        max_alternatives=0)


{
  "results": [
    {
      "final": true,
      "alternatives": [
        {
          "transcript": "fate whispers to the warrior ",
          "confidence": 0.74
        }
      ]
    },
    {
      "final": true,
      "alternatives": [
        {
          "transcript": "storms coming ",
          "confidence": 0.66
        }
      ]
    },
    {
      "final": true,
      "alternatives": [
        {
          "transcript": "and the warrior whispers back ",
          "confidence": 0.7
        }
      ]
    },
    {
      "final": true,
      "alternatives": [
        {
          "transcript": "I am the star ",
          "confidence": 0.48
        }
      ]
    },
    {
      "final": true,
      "alternatives": [
        {
          "transcript": "%HESITATION ",
          "confidence": 0.99
        }
      ]
    },
    {
      "final": true,
      "alternatives": [
        {
          "transcript": "good evening Mr out the anarchist Solomon like ",
          "confidence": 0.66
     

In [5]:
# FETCHING ONLY DIALOGS FROM THE OUTPUT
import json
file_path="FILE NAME TO WHICH RESULTS WERE WRITTEN.json"


with open(file_path, "r") as read_file:
    data = json.load(read_file)

fetched_dialogs=[]
for i in range(0,len(data["results"])):
    fetched_dialogs.append(data["results"][i]["alternatives"][0]["transcript"])
    
print(fetched_dialogs)

['fate whispers to the warrior ', 'storms coming ', 'and the warrior whispers back ', 'I am the star ', '%HESITATION ', 'good evening Mr out the anarchist Solomon like ', 'since you captured him to ']


In [8]:
#COMPARING FETCHED DIALOGS FROM THE SOURCE WITH THE DIALOGS STORED IN OUR DATABASE

from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()



print(similar(fetched_dialogs[5],'good evening Mr hunt the anarchist Solomon lane'))

0.9148936170212766
